In [3]:
import arcpy
import requests
import json
import pprint
import zipfile
import arcpy
import pandas as pd
import os
import numpy as np
from datetime import timedelta
from datetime import datetime

# Request Station Data

In [4]:
# Adding my token to a variable
Token = "EWfeohdvgAYBUAOZMoPgKNBjgaFAjMXS"

In [5]:
# Writing my params for which stations to choose
station_params = {'datasetid': 'GHCND', 'startdate': (datetime.now() - timedelta(days=7)).strftime('%Y-%m-%d'),
               'enddate': datetime.now().strftime('%Y-%m-%d'), 'locationid': 'FIPS:27', 'datatypeid': ['TMIN', 'TMAX'], 'limit':'1000'}

In [6]:
# Adding the base url to a variable - Getting Stations
base_url = 'https://www.ncdc.noaa.gov/cdo-web/api/v2/stations'

In [7]:
# Making the request for specific stations
stations_req = requests.get(base_url, params=station_params, headers={'token':Token})
stations_req

<Response [200]>

In [8]:
# Translating the request into JSON format and printing
stations_j = stations_req.json()

# Creating a Point Feature Class for the Available Stations

In [9]:
# Creating an empty feature class of type Point
stations = stations_j['results']
workspace = arcpy.env.workspace
arcpy.CreateFeatureclass_management(workspace,"stations","POINT","","DISABLED","DISABLED",arcpy.SpatialReference(4269))

<Result 'C:\\Users\\ecava\\Documents\\ArcGIS\\Projects\\Lab2\\Lab2.gdb\\stations'>

In [10]:
# Adding fields to the feature class
arcpy.AddField_management("stations","id","TEXT","","",100)
arcpy.AddField_management("stations","name","TEXT","","",100)
arcpy.AddField_management("stations","mindate","DATE")
arcpy.AddField_management("stations","maxdate","DATE")

<Result 'stations'>

In [11]:
# Creating a cursor to add the info to the feature class
cur = arcpy.da.InsertCursor("stations", ["id", "name", "mindate", "maxdate", "Shape@"])

for s in stations:
   
    point = arcpy.Point(s['longitude'],s['latitude'])
    row = [s["id"],s["name"],s["mindate"],s["maxdate"],point]
   
    cur.insertRow(row)
   
del cur

# Request Temperature Data

In [12]:
# Creating a list of the stations that can be looped through to download data
stations_list = []
for station in stations:
    stations_list.append(station['id'])

In [13]:
# Base URL for requesting temperature data
data_req_url = 'https://www.ncdc.noaa.gov/cdo-web/api/v2/data?'

## Daily Minimum Temps at Each Station

In [14]:
# Creating a dictionary of the stations to hold Min temp data for each station
stations_dict_min = dict.fromkeys(stations_list)

In [15]:
# Looping through the list of stations 
# and making requests for the min temp for each station
# for the time period.
for s_name in stations_list:
    min_params = {'datasetid': 'GHCND', 'datatypeid': 'TMIN', 'units': 'standard', 'limit': '7', 'stationid': s_name, 'startdate': (datetime.now() - timedelta(days=7)).strftime('%Y-%m-%d'), 'enddate': datetime.now().strftime('%Y-%m-%d')}
    # make the api call
    min_req = requests.get(data_req_url, params = min_params, headers = {'token':Token})
    # load the api response as a json
    min_d = min_req.json()
    stations_dict_min[s_name] = min_d

In [17]:
# Creating an Array of Arrays by looping through
# The min-temp json and grabbing the date, temp, and station id and
# Appending them to a list then appending that to the 'all-station array'
min_all_station_array = []
for station in stations_dict_min:
    for results in stations_dict_min[station]['results']:
        station_array = []
        station_array.append(results['date'])
        station_array.append(results['value'])
        station_array.append(results['station'])
        min_all_station_array.append(station_array)

In [18]:
# Creating a dataframe from the array and naming the columns
all_min_temps = pd.DataFrame(min_all_station_array, columns = ['datetime', 'min_temp', 'station'])

In [19]:
# Pivoting the dataframe so that the index is station id and each day is a column
min_station_data = all_min_temps.pivot(index = 'station', columns = 'datetime', values = 'min_temp')
min_station_data

datetime,2022-05-04T00:00:00,2022-05-05T00:00:00,2022-05-06T00:00:00,2022-05-07T00:00:00,2022-05-08T00:00:00,2022-05-09T00:00:00
station,,,,,,
GHCND:CA006020559,30.0,31.0,36.0,NaN,NaN,NaN
GHCND:USC00210018,NaN,NaN,NaN,47.0,50.0,NaN
GHCND:USC00210075,37.0,44.0,46.0,46.0,48.0,50.0
GHCND:USC00210287,37.0,41.0,41.0,47.0,51.0,NaN
GHCND:USC00210355,33.0,NaN,44.0,44.0,50.0,48.0
...,...,...,...,...,...,...
GHCND:USW00094960,33.0,41.0,39.0,51.0,NaN,NaN
GHCND:USW00094961,33.0,35.0,43.0,50.0,NaN,NaN
GHCND:USW00094963,39.0,42.0,43.0,48.0,NaN,NaN


## Daily Maximum Temps at Each Station

In [20]:
# Creating a dictionary of the stations to hold data for each station
stations_dict_max = dict.fromkeys(stations_list)

In [21]:
# Looping through the list of stations and making requests for the max temp for each station
# for the time period.
for s_name in stations_list:
    max_params = {'datasetid': 'GHCND', 'datatypeid': 'TMAX', 'units': 'standard', 'limit': '7', 'stationid': s_name, 'startdate': (datetime.now() - timedelta(days=7)).strftime('%Y-%m-%d'), 'enddate': datetime.now().strftime('%Y-%m-%d')}
    # make the api call
    max_req = requests.get(data_req_url, params = max_params, headers = {'token':Token})
    # load the api response as a json
    max_d = max_req.json()
    stations_dict_max[s_name] = max_d

In [22]:
max_all_station_array = []
for station in stations_dict_max:
    for results in stations_dict_max[station]['results']:
        station_array = []
        station_array.append(results['date'])
        station_array.append(results['value'])
        station_array.append(results['station'])
        max_all_station_array.append(station_array)

In [23]:
df_max = pd.DataFrame(max_all_station_array, columns = ['datetime', 'max_temp', 'station'])

In [24]:
# Pivoting the dataframe so that the index is station id and each day is a column
df_max_pivot_station = df_max.pivot(index = 'station', columns = 'datetime', values = 'max_temp')
df_max_pivot_station

datetime,2022-05-04T00:00:00,2022-05-05T00:00:00,2022-05-06T00:00:00,2022-05-07T00:00:00,2022-05-08T00:00:00,2022-05-09T00:00:00
station,,,,,,
GHCND:CA006020559,63.0,70.0,NaN,NaN,NaN,NaN
GHCND:USC00210018,NaN,NaN,NaN,70.0,73.0,NaN
GHCND:USC00210075,56.0,62.0,58.0,68.0,72.0,60.0
GHCND:USC00210287,62.0,62.0,72.0,74.0,72.0,NaN
GHCND:USC00210355,55.0,NaN,59.0,66.0,70.0,59.0
...,...,...,...,...,...,...
GHCND:USW00094960,65.0,66.0,73.0,75.0,NaN,NaN
GHCND:USW00094961,64.0,69.0,73.0,75.0,NaN,NaN
GHCND:USW00094963,64.0,65.0,72.0,75.0,NaN,NaN


## Joining both min and max to one df > csv

In [66]:
all_data = pd.merge(df_pivot_station, df_max_pivot_station, 'outer', on = 'station')

NameError: name 'df_pivot_station' is not defined

In [ ]:
all_data.to_csv(r"C:\Users\ecava\Documents\ArcGIS\Projects\Lab2\temp_data.csv")

In [ ]:
# Join
arcpy.management.AddJoin("stations", "id", "temp_data.csv", "station", "KEEP_ALL", "NO_INDEX_JOIN_FIELDS")

In [ ]:
# Arcpy describe or pd columns
days_list = list(all_data.columns)

# Growing Degree Days

For corn, the degree-day accumulation base is 50.

In [25]:
# Creating a list of the available days to loop through
days_list = list(min_station_data.columns)
print(days_list)
# a = days_list[0]
# min_station_data[a].to_list()

['2022-05-04T00:00:00', '2022-05-05T00:00:00', '2022-05-06T00:00:00', '2022-05-07T00:00:00', '2022-05-08T00:00:00', '2022-05-09T00:00:00']


In [26]:
# Creating a list of stations for use as a join column
stations_list = df_max_pivot_station.index.to_list()

## Method 1: Temperature averaging

Simply take mean of each day and subtract the degree-day accumulation base (50). If the resulting number is negative, the GDD value is 0.

In [27]:
a = 0
for day in days_list:
    # Instantiating an empty dataframe
    day_df = pd.DataFrame()
    # Creating a stations column
    day_df['station'] = stations_list
    # Creating a minimum temp column
    day_df['min'] = min_station_data[day].to_list()
#     Creating a maximum temp column
    day_df['max'] = df_max_pivot_station[day].to_list()
    # Creating the mean column by adding min and max and dividing by 2
    day_df['mean'] = day_df['min'] + day_df['max']
    day_df['mean'] = day_df['mean'].astype(float)
    day_df['mean'] = day_df['mean'].div(2)
    
    day_df['mean'] = day_df['mean'].replace(np.nan, 0)

    # Calculating the GDD values for each station by subracting base value
    day_df['gdd'] = day_df['mean'] - 50
    
    gdd_list = day_df['gdd'].to_list()
#     print(gdd_list)
    i = 0
    while i < len(gdd_list):
        if gdd_list[i] < 0:
            gdd_list[i] = 0
        i+=1
    
    day_df['gdd'] = gdd_list
    
    # Creating a save name for that day's data
    save_name = 'gdd'+ str(a)
    a += 1
    # Creating a CSV from the day's data
    day_df.to_csv(r"C:\Users\ecava\Documents\ArcGIS\Projects\Lab2\\" + save_name + ".csv")
    
    # Joining the CSV to the station points
    arcpy.management.AddJoin("stations", "id", save_name+".csv", "station", "KEEP_ALL", "NO_INDEX_JOIN_FIELDS")
    
    # Selecting all data
    arcpy.management.SelectLayerByAttribute("stations", "NEW_SELECTION", "OBJECTID IS NOT NULL", None)
    
    # Exporting a copy of the stations joined to that day's data
    arcpy.management.CopyFeatures("stations", r"C:\Users\ecava\Documents\ArcGIS\Projects\Lab2\Lab2.gdb\\"+ save_name, '', None, None, None)
    
    # Remove column of data from the stations feature class
    arcpy.RemoveJoin_management('stations')

In [28]:
len(days_list)

6

In [29]:
# Run interpolation on the resulting point layer
avg_rmse_list = []
n = 0
while n < len(days_list):
    n = str(n)
    arcpy.ga.GlobalPolynomialInterpolation("gdd"+n, "gdd"+n+"_csv_gdd", "gdd"+n+"i", None, 0.02130268, 1, None)
    cvResult = arcpy.ga.CrossValidation("gdd"+n+"i", r"C:\Users\ecava\Documents\ArcGIS\Projects\Lab2\Lab2.gdb\gdd"+n+"icv")
    avg_rmse_list.append(cvResult.rootMeanSquare)
    n = int(n)
    n+=1
avg_rmse_list

['0.2727542028493769', '1.4024861317327315', '2.4471635711131228', '3.7586281798269994', '4.23678900228468', '4.670311807926968']

## Method 2: Modified Average

- If the low temperature of the day is below your crop or pests' base value, use the base temperature during your calculations.
- If the high temperature for the day is above 86 degrees Fahrenheit, use this value instead of the actual high temperature.
- If the mean temperature is at or below the base temperature for a crop or pest of interest, then the GDD value is zero. 
- If the mean temperature is above the base temperature, then the GDD equals the value of the mean temperature minus the base temperature.

In [30]:
modavg_rmse_list = []
a = 0
for day in days_list:
    # Instantiating empty dataframes
    day_df = pd.DataFrame()
    min_df = pd.DataFrame()
    max_df = pd.DataFrame()
    
    # Creating a stations column
    day_df['station'] = stations_list
    
    # Creating a minimum temp column
    min_df['min'] = min_station_data[day].to_list()
    # Checking if min temp is below base temp and changing it if it is
    min_df[min_df < 50] = 50
    
    # Adding new min col to main day df
    day_df['min'] = min_df['min'].to_list()
    
    # Creating a maximum temp column
    max_df['max'] = df_max_pivot_station[day].to_list()
    # Checking if max temp is above max temp and changing it if it is
    max_df[max_df > 86] = 86
    # Adding new min col to main day df
    day_df['max'] = max_df['max'].to_list()
    
    # Creating the mean column by adding min and max and dividing by 2
    day_df['mean'] = day_df['min'] + day_df['max']
    day_df['mean'] = day_df['mean'].astype(float)
    day_df['mean'] = day_df['mean'].div(2)
    
    day_df['mean'] = day_df['mean'].replace(np.nan, 0)

    # Calculating the GDD values for each station by subracting base value
    day_df['gdd'] = day_df['mean'] - 50
    
    gdd_list = day_df['gdd'].to_list()
    # Ensuring that if the value is negative, the value is 0 
    i = 0
    while i < len(gdd_list):
        if gdd_list[i] < 0:
            gdd_list[i] = 0
        i+=1
    
    day_df['gdd'] = gdd_list
    
    # Creating a save name for that day's data
    save_name = 'modgdd' + str(a)
    a += 1
    
    # Creating a CSV from the day's data
    day_df.to_csv(r"C:\Users\ecava\Documents\ArcGIS\Projects\Lab2\\" + save_name + ".csv")
    
    # Joining the CSV to the station points
    arcpy.management.AddJoin("stations", "id", save_name+".csv", "station", "KEEP_ALL", "NO_INDEX_JOIN_FIELDS")
    
    # Selecting all data
    arcpy.management.SelectLayerByAttribute("stations", "NEW_SELECTION", "OBJECTID IS NOT NULL", None)
    
    # Exporting a copy of the stations joined to that day's data
    arcpy.management.CopyFeatures("stations", r"C:\Users\ecava\Documents\ArcGIS\Projects\Lab2\Lab2.gdb\\"+ save_name, '', None, None, None)
    
    # Remove column of data from the stations feature class
    arcpy.RemoveJoin_management('stations')

In [31]:
# modgdd4_csv_gdd
n = 0
while n < len(days_list):
    n = str(n)
    arcpy.ga.GlobalPolynomialInterpolation("modgdd"+n, "modgdd"+n+"_csv_gdd", "modgdd"+n+"i", None, 0.02130268, 1, None)
    cvResult = arcpy.ga.CrossValidation("modgdd"+n+"i", r"C:\Users\ecava\Documents\ArcGIS\Projects\Lab2\Lab2.gdb\modgdd"+n+"icv")
    modavg_rmse_list.append(cvResult.rootMeanSquare)
    n = int(n)
    n+=1
modavg_rmse_list

['2.454474466364828', '3.08435083695795', '3.2420933935500633', '3.618280860844914', '4.4458123558877745', '4.6750963656350635']

## Method 3: Baskerville-Emin method

Consists of the following steps:
1. If max temperature is less than base temp, GDD is 0
3. If min temp is greater than or equal to the base temp: GDD = average - base temp
4. If 2 is not true:
    - TAVE = ((max+min)/2)
    - BASE = 50
    - W = (Max - Min)/2
    - A = Arcsin((BASE-TAVE)/W)
    - GDD = ((W*Cos(A))-((BASE-TAVE)*(3.14/2)-A)))/3.14

In [32]:
be_rmse_list = []
a = 0
for day in days_list:
    # Instantiating an empty dataframe
    day_df = pd.DataFrame()
    # Creating a stations column
    day_df['station'] = stations_list
    # Creating a minimum temp column
    day_df['min'] = min_station_data[day].to_list()
    day_df['min'] = day_df['min'].replace(np.nan, 0)
    # Creating a maximum temp column
    day_df['max'] = df_max_pivot_station[day].to_list()
    day_df['max'] = day_df['max'].replace(np.nan, 0)
    
    gdd_list = []
    base_temp = 50
    for index, row in day_df.iterrows():
        if row['max'] < base_temp:
            gdd_list.append(0)
        else:
            gdd_list.append('not less than')
            
    nlt_indices = []
    for i in range(len(gdd_list)):
        if gdd_list[i] == 'not less than':
            nlt_indices.append(i)
    
    for n in nlt_indices:
        avg = ((day_df['min'][n]+ day_df['max'][n])/2)
        if day_df['min'][n] >= 50:
            gdd_list[n] = (avg - 50)
        else:
            W = ((day_df['max'][n] - day_df['min'][n])/2)
            A = np.arcsin((base_temp-avg)/W)
            cos = np.cos(A)
            gdd = (W*np.cos(A))-((base_temp-avg)*(1.57)-A)/3.14
            gdd_list[n] = gdd
    day_df['gdd'] = gdd_list
    day_df['gdd'] = day_df['gdd'].replace(np.nan, 0)
    # Creating a save name for that day's data
    save_name = 'be' + str(a)
    a += 1
    # Creating a CSV from the day's data
    day_df.to_csv(r"C:\Users\ecava\Documents\ArcGIS\Projects\Lab2\\" + save_name + ".csv")
    
    # Joining the CSV to the station points
    arcpy.management.AddJoin("stations", "id", save_name+".csv", "station", "KEEP_ALL", "NO_INDEX_JOIN_FIELDS")
    
    # Selecting all data
    arcpy.management.SelectLayerByAttribute("stations", "NEW_SELECTION", "OBJECTID IS NOT NULL", None)
    
    # Exporting a copy of the stations joined to that day's data
    arcpy.management.CopyFeatures("stations", r"C:\Users\ecava\Documents\ArcGIS\Projects\Lab2\Lab2.gdb\\"+ save_name, '', None, None, None)
    
    # Remove column of data from the stations feature class
    arcpy.RemoveJoin_management('stations')

In [33]:
n = 0
while n < len(days_list):
    n = str(n)
    arcpy.ga.GlobalPolynomialInterpolation("be"+n, "be"+n+"_csv_gdd", "be"+n+"i", None, 0.02130268, 1, None)
    cvResult = arcpy.ga.CrossValidation("be"+n+"i", r"C:\Users\ecava\Documents\ArcGIS\Projects\Lab2\Lab2.gdb\be"+n+"icv")
    be_rmse_list.append(cvResult.rootMeanSquare)
    n = int(n)
    n+=1

# Compare RMSE between each model

In [34]:
# Compare the list of daily RMSE for each model

In [36]:
# Comparing the three lists of rmses to see who has the lowest rmse on each day
winner_list = []
for r in be_rmse_list:
    ind = be_rmse_list.index(r)
    if r < modavg_rmse_list[ind]:
        winner_list.append('be')
    if r > modavg_rmse_list[ind]:
        winner_list.append('mod')
    n = 0
    newind = winner_list[n]
    if newind > avg_rmse_list[n]:
        winner_list[n] = 'avg'
        n+=1

In [37]:
countbe = winner_list.count('be')
countmod = winner_list.count('mod')
countavg = winner_list.count('avg')
string = []
if countbe > countmod:
    if countbe > countavg:
        int_string = 'be'
if countmod > countavg:
    int_string = 'modgdd'
else:
    int_string = 'gdd'

In [38]:
print(int_string)

modgdd


# Push (max of 7) gdd maps to postgis

In [104]:
# Automatically send shapefile of GDD using best model to POSTGIS database

In [39]:
import psycopg2

In [49]:
n = 0
while n < len(days_list):
    feature_class = r"C:\Users\ecava\Documents\ArcGIS\Projects\Lab2\Lab2.gdb\\"+int_string+ str(n)
    field_names = [f.name for f in arcpy.ListFields(feature_class)]
    
    newlist = field_names[8:]
    fields_to_insert = ['Shape@WKT', 'stations_id']
    fields = fields_to_insert + newlist
    print(fields)

    i = 2
    string_add = []
    while i < len(fields)-1:
        string_add.append(fields[i] + ' FLOAT, ')
        i +=1
    string_add = ' '.join(string_add)
    string_fields = string_add + fields[(len(fields)-1)] + ' FLOAT)'
    print(string_fields)
    
    a = 1
    end = []
    signs = ['%s']
    while a <len(fields)-1:
        end.append(fields[a])
        signs.append('%s')
        a+=1

    end_2 = ', '.join(end)
    end_3 = end_2 + ', '+fields[(len(fields)-1)]

    signs_2 = ', '.join(signs)
    signs_3 = signs_2 + ',%s)'

    work_pls = end_3 + ') VALUES ('+ signs_3
    
    def bulkInsert(records, table_name):
        conn=psycopg2.connect("dbname='postgres' user='postgres' host='34.72.222.158' password='postgres'")
        cursor = conn.cursor()
        query = "INSERT INTO " + table_name + ' (Shape, ' + work_pls
        # executemany() to insert multiple rows
        result = cursor.executemany(query, records)
        conn.commit()

    conn=psycopg2.connect("dbname='postgres' user='postgres' host='34.72.222.158' password='postgres'")
    cursor=conn.cursor()
    cursor.execute("Drop Table if exists GDD_BestMethod"+str(n))
    string = 'CREATE TABLE GDD_BestMethod'+str(n)+' (Shape geometry, stations_id text, '
    string_final = string + string_fields
    
    cursor.execute(string_final)
    conn.commit()
    cursor = arcpy.da.SearchCursor(feature_class, fields)
    point_list = []
    for row in cursor:
        pt_tuple = ()
        i = 0
        while i < len(fields):
            pt_tuple = pt_tuple + (row[i],)
            i+=1
        point_list.append(pt_tuple)
    # Connect to database
    bulkInsert(point_list, "GDD_BestMethod"+str(n))
    n +=1

['Shape@WKT', 'stations_id', 'modgdd0_csv_min', 'modgdd0_csv_max', 'modgdd0_csv_mean', 'modgdd0_csv_gdd']
modgdd0_csv_min FLOAT,  modgdd0_csv_max FLOAT,  modgdd0_csv_mean FLOAT, modgdd0_csv_gdd FLOAT)
['Shape@WKT', 'stations_id', 'modgdd1_csv_min', 'modgdd1_csv_max', 'modgdd1_csv_mean', 'modgdd1_csv_gdd']
modgdd1_csv_min FLOAT,  modgdd1_csv_max FLOAT,  modgdd1_csv_mean FLOAT, modgdd1_csv_gdd FLOAT)
['Shape@WKT', 'stations_id', 'modgdd2_csv_min', 'modgdd2_csv_max', 'modgdd2_csv_mean', 'modgdd2_csv_gdd']
modgdd2_csv_min FLOAT,  modgdd2_csv_max FLOAT,  modgdd2_csv_mean FLOAT, modgdd2_csv_gdd FLOAT)
['Shape@WKT', 'stations_id', 'modgdd3_csv_min', 'modgdd3_csv_max', 'modgdd3_csv_mean', 'modgdd3_csv_gdd']
modgdd3_csv_min FLOAT,  modgdd3_csv_max FLOAT,  modgdd3_csv_mean FLOAT, modgdd3_csv_gdd FLOAT)
['Shape@WKT', 'stations_id', 'modgdd4_csv_min', 'modgdd4_csv_max', 'modgdd4_csv_mean', 'modgdd4_csv_gdd']
modgdd4_csv_min FLOAT,  modgdd4_csv_max FLOAT,  modgdd4_csv_mean FLOAT, modgdd4_csv_gdd FL

# Evapotranspiration

## Kharrufa

ET = 0.34pTa^(1.3)
where ET is the Kharrufa potential evapotranspiration (in mm/month) Ta is mean temperature in °C, p is percentage of total daytime hours for the period used (daily or monthly) out of total daytime hours of the year.

## Percentage of daytime hours and latitude: Setup

In [50]:
# initialize look-up dictionary
pct_daylight_hrs = [['01',40,45,0.22],['02',40,45,0.24],['03',40,45,0.27], ['04',40,45,0.3], ['05',40,45,0.32],['06',40,45,0.34],['07',40,45,0.33], ['08',40,45,0.31],['09',40,45,0.28],['10',40,45,0.25],['11',40,45,0.22],['12',40,45,0.21],['01',45,50,0.2],['02',45,50,0.23],['03',45,50,0.27],['04',45,50,0.3],['05',45,50,0.34],['06',45,50,0.35],['07',45,50,0.34],['08',45,50,0.32],['09',45,50,0.28],['10',45,50,0.24],['11',45,50,0.21],['12',45,50,0.2]]

# Create the pandas DataFrame
lookup_df = pd.DataFrame(pct_daylight_hrs, columns = ['month', 'min_lat', 'max_lat', 'daylight_pct'])

In [51]:
# First Add New Field to stations to hold latitude
with arcpy.EnvManager(scratchWorkspace=r"C:\Users\ecava\Documents\ArcGIS\Projects\Lab2\Lab2.gdb", workspace=r"C:\Users\ecava\Documents\ArcGIS\Projects\Lab2\Lab2.gdb"):
    arcpy.management.AddField("stations", "lat", "FLOAT", None, None, None, '', "NULLABLE", "NON_REQUIRED", '')

In [52]:
# Calculate Geometry of Latitude of stations
arcpy.management.CalculateGeometryAttributes("stations", "lat POINT_Y", '', '', None, "SAME_AS_INPUT")

<Result 'stations'>

In [53]:
# Turn table into excel
arcpy.conversion.TableToExcel("stations", r"C:\Users\ecava\Documents\ArcGIS\Projects\Lab2\stations_TableToExcel.xlsx", "NAME", "CODE")

<Result 'C:\\Users\\ecava\\Documents\\ArcGIS\\Projects\\Lab2\\stations_TableToExcel.xlsx'>

## Creating Layers (Max 7 days)

In [54]:
# Turn excel into dataframe to lookup percent
lat_station_df = pd.read_excel(r"C:\Users\ecava\Documents\ArcGIS\Projects\Lab2\stations_TableToExcel.xlsx")

In [80]:
k_rmse_list = []
n = 0
for day in days_list:
    
    # Instantiating an empty dataframes
    day_df = pd.DataFrame()
    min_temp_c = pd.DataFrame()
    max_temp_c = pd.DataFrame()
    day_df['station'] = stations_list
    # Creating a minimum temp column
    # Create a min temp df from that day
    # Convert Temps to Celsius = (°F – 32) x 5/9
    min_orig = min_station_data[day].to_list()
    min_temp_c['min'] = min_orig
    min_temp_c = min_temp_c.sub(32)
    min_temp_c = min_temp_c.div(9)
    min_temp_c = min_temp_c.mul(5)
    
    # Add min column to day_df
    day_df['min'] = min_temp_c['min'].to_list()
    
    # Creating a maximum temp column
    # Create a max temp df from that day
    # Convert Temps to Celsius
    max_orig = df_max_pivot_station[day].to_list()
    max_temp_c['max'] = max_orig
    max_temp_c = max_temp_c.sub(32)
    max_temp_c = max_temp_c.div(9)
    max_temp_c = max_temp_c.mul(5)
    
    # Add max column to day_df
    day_df['max'] = max_temp_c['max'].to_list()
    day_df = day_df.replace(np.nan, 0)
    
    # Creating the mean column by adding min and max and dividing by 2
    day_df['mean'] = day_df['min'] + day_df['max']
    
    # Calculating Mean column
    mean_in_eq = pd.DataFrame()
    mean_in_eq['mean'] = day_df['mean'].to_list()
    mean_in_eq = mean_in_eq.div(2)
    mean_in_eq = mean_in_eq.pow(1.3)
    mean_in_eq = mean_in_eq.mul(0.34)
    # Now ET = p*day_df['mean']
    day_df['mean'] = mean_in_eq['mean'].to_list()
    
    # Add latitude column to day_df
    day_df['lat'] = lat_station_df.loc[:, ['lat']]

    # Instantiate empty percent daylight list
    p_list = []
    month = day[5:7]
    temp_month = lookup_df.loc[lookup_df['month'] == month]
    
    for index, row in day_df.iterrows():
        if row['lat'] < 45:
            v = temp_month.index[temp_month['max_lat'] == 45].to_list()
            v = v[0]
            row = lookup_df.iloc[[v]]
            pct_list = row['daylight_pct'].to_list()
            p = pct_list[0]
            p_list.append(p)
        else:
            v = temp_month.index[temp_month['max_lat'] == 50].to_list()
            v = v[0]
            row = lookup_df.iloc[[v]]
            pct_list = row['daylight_pct'].to_list()
            p =pct_list[0]
            p_list.append(p)
    day_df['p'] = p_list
    day_df['et'] = day_df['p'] * day_df['mean']
    
    # Creating a save name for that day's data
    save_name = 'k_et'+ str(n)

    # Creating a CSV from the day's data
    day_df.to_csv(r"C:\Users\ecava\Documents\ArcGIS\Projects\Lab2\\" + save_name + ".csv")
    
    # Joining the CSV to the station points
    arcpy.management.AddJoin("stations", "id", save_name+".csv", "station", "KEEP_ALL", "NO_INDEX_JOIN_FIELDS")
    
    # Selecting all data
    arcpy.management.SelectLayerByAttribute("stations", "NEW_SELECTION", "OBJECTID IS NOT NULL", None)
    
    # Exporting a copy of the stations feature class joined to that day's data
    arcpy.management.CopyFeatures("stations", r"C:\Users\ecava\Documents\ArcGIS\Projects\Lab2\Lab2.gdb\\"+ save_name, '', None, None, None)
    
    # Remove column of data from the stations feature class
    arcpy.RemoveJoin_management('stations')

    n += 1

In [82]:
n = 0
while n < len(days_list):
    n = str(n)
    arcpy.ga.GlobalPolynomialInterpolation("k_et"+n, "k_et"+n+"_csv_et", "k_et"+n+"i", None, 0.02130268, 1, None)
    cvResult = arcpy.ga.CrossValidation("k_et"+n+"i", r"C:\Users\ecava\Documents\ArcGIS\Projects\Lab2\Lab2.gdb\k_et"+n+"icv")
    k_rmse_list.append(cvResult.rootMeanSquare)
    n = int(n)
    n+=1
print(k_rmse_list)

['0.6524917106517513', '0.8774503247903924', '0.8909178177633358', '1.1995049885114883', '1.529571767424264', '1.7674210004015922']


## Blaney Criddle

ET = kp(0.46)Ta + 8.13

where ET is evapotranspiration from the reference crop (in mm) for the period in which p is expressed, Ta is mean temperature in °C, p is percentage of total daytime hours for the period used (daily or monthly) out of total daytime hours of the year (365 x 12), and k is a monthly consumptive use coefficient, depending on vegetation type, location and season (using 0.8).

ET = (0.8)* p * (0.46)* Ta + 8.13

In [85]:
bc_rmse_list = []
n = 0
for day in days_list:
    
    # Instantiating an empty dataframes
    day_df = pd.DataFrame()
    min_temp_c = pd.DataFrame()
    max_temp_c = pd.DataFrame()
    day_df['station'] = stations_list
    # Creating a minimum temp column
    # Create a min temp df from that day
    # Convert Temps to Celsius = (°F – 32) x 5/9
    min_orig = min_station_data[day].to_list()
    min_temp_c['min'] = min_orig
    min_temp_c = min_temp_c.sub(32)
    min_temp_c = min_temp_c.div(9)
    min_temp_c = min_temp_c.mul(5)
    
    # Add min column to day_df
    day_df['min'] = min_temp_c['min'].to_list()
    
    # Creating a maximum temp column
    # Create a max temp df from that day
    # Convert Temps to Celsius
    max_orig = df_max_pivot_station[day].to_list()
    max_temp_c['max'] = max_orig
    max_temp_c = max_temp_c.sub(32)
    max_temp_c = max_temp_c.div(9)
    max_temp_c = max_temp_c.mul(5)
    
    # Add max column to day_df
    day_df['max'] = max_temp_c['max'].to_list()
    day_df = day_df.replace(np.nan, 0)
    
    # Creating the mean column by adding min and max and dividing by 2
    day_df['mean'] = day_df['min'] + day_df['max']
    
    # Calculating Mean column
    mean_in_eq = pd.DataFrame()
    mean_in_eq['mean'] = day_df['mean'].to_list()
    mean_in_eq = mean_in_eq.div(2)
    mean_in_eq = mean_in_eq.mul(0.8)
    mean_in_eq = mean_in_eq.mul(0.46)
    # Now ET = p*day_df['mean']
    day_df['mean'] = mean_in_eq['mean'].to_list()
    
    
# Add latitude column to day_df
    day_df['lat'] = lat_station_df.loc[:, ['lat']]

    # Instantiate empty percent daylight list
    p_list = []
    month = day[5:7]
    temp_month = lookup_df.loc[lookup_df['month'] == month]
    
    for index, row in day_df.iterrows():
        if row['lat'] < 45:
            v = temp_month.index[temp_month['max_lat'] == 45].to_list()
            v = v[0]
            row = lookup_df.iloc[[v]]
            pct_list = row['daylight_pct'].to_list()
            p = pct_list[0]
            p_list.append(p)
        else:
            v = temp_month.index[temp_month['max_lat'] == 50].to_list()
            v = v[0]
            row = lookup_df.iloc[[v]]
            pct_list = row['daylight_pct'].to_list()
            p =pct_list[0]
            p_list.append(p)
    day_df['p'] = p_list
    day_df['et'] = day_df['p'] * day_df['mean']
    
    day_df['et'] = day_df['et'] + 8.13
    
    # Creating a save name for that day's data
    save_name = 'bc' + str(n)
    
    # Creating a CSV from the day's data
    day_df.to_csv(r"C:\Users\ecava\Documents\ArcGIS\Projects\Lab2\\" + save_name + ".csv")
    
    # Joining the CSV to the station points
    arcpy.management.AddJoin("stations", "id", save_name+".csv", "station", "KEEP_ALL", "NO_INDEX_JOIN_FIELDS")
    
    # Selecting all data
    arcpy.management.SelectLayerByAttribute("stations", "NEW_SELECTION", "OBJECTID IS NOT NULL", None)
    
    # Exporting a copy of the stations feature class joined to that day's data
    arcpy.management.CopyFeatures("stations", r"C:\Users\ecava\Documents\ArcGIS\Projects\Lab2\Lab2.gdb\\"+ save_name, '', None, None, None)
    
        # Remove column of data from the stations feature class
    arcpy.RemoveJoin_management('stations')
    n +=1

In [86]:
n = 0
while n < len(days_list):
    n = str(n)
    arcpy.ga.GlobalPolynomialInterpolation("bc"+n, "bc"+n+"_csv_et", "bc"+n+"i", None, 0.02130268, 1, None)
    cvResult = arcpy.ga.CrossValidation("bc"+n+"i", r"C:\Users\ecava\Documents\ArcGIS\Projects\Lab2\Lab2.gdb\bc"+n+"icv")
    bc_rmse_list.append(cvResult.rootMeanSquare)
    n = int(n)
    n+=1
print(bc_rmse_list)

['0.35285523032883814', '0.44741062496811623', '0.4257967449468412', '0.5568246157814677', '0.7224569952365121', '0.844490656206078']


# Compare RMSE between each model

In [87]:
# Comparing the three lists of rmses to see who has the lowest rmse on each day
et_winner_list = []
for r in k_rmse_list:
    ind = k_rmse_list.index(r)
    if r < bc_rmse_list[ind]:
        et_winner_list.append('k')
    else:
        et_winner_list.append('bc')

In [88]:
countk = et_winner_list.count('k')
countbc = et_winner_list.count('bc')

et_string = []
if countk > countbc:
    et_string = 'k_et'
else:
    et_string = 'bc'

In [89]:
print(et_string)

bc


# Push (max of 7) Evapotranspiration Maps to Postgis

In [90]:
# Change the string to the winning model
# Change the number of columns to exclude depending on the number of columns before your main data
# Change the table name to something descriptive of the index of interest here
n = 0
while n < len(days_list):
    feature_class = r"C:\Users\ecava\Documents\ArcGIS\Projects\Lab2\Lab2.gdb\\"+et_string+ str(n)
    field_names = [f.name for f in arcpy.ListFields(feature_class)]
    
    newlist = field_names[9:]
    fields_to_insert = ['Shape@WKT', 'stations_id']
    fields = fields_to_insert + newlist
    print(fields)

    i = 2
    string_add = []
    while i < len(fields)-1:
        string_add.append(fields[i] + ' FLOAT, ')
        i +=1
    string_add = ' '.join(string_add)
    string_fields = string_add + fields[(len(fields)-1)] + ' FLOAT)'
    print(string_fields)
    
    a = 1
    end = []
    signs = ['%s']
    while a <len(fields)-1:
        end.append(fields[a])
        signs.append('%s')
        a+=1

    end_2 = ', '.join(end)
    end_3 = end_2 + ', '+fields[(len(fields)-1)]

    signs_2 = ', '.join(signs)
    signs_3 = signs_2 + ',%s)'

    work_pls = end_3 + ') VALUES ('+ signs_3
    
    def bulkInsert(records, table_name):
        conn=psycopg2.connect("dbname='postgres' user='postgres' host='34.72.222.158' password='postgres'")
        cursor = conn.cursor()
        query = "INSERT INTO " + table_name + ' (Shape, ' + work_pls
        # executemany() to insert multiple rows
        result = cursor.executemany(query, records)
        conn.commit()

    conn=psycopg2.connect("dbname='postgres' user='postgres' host='34.72.222.158' password='postgres'")
    cursor=conn.cursor()
    cursor.execute("Drop Table if exists ET_BestMethod"+str(n))
    string = 'CREATE TABLE ET_BestMethod'+str(n)+' (Shape geometry, stations_id text, '
    string_final = string + string_fields
    
    cursor.execute(string_final)
    conn.commit()
    cursor = arcpy.da.SearchCursor(feature_class, fields)
    point_list = []
    for row in cursor:
        pt_tuple = ()
        i = 0
        while i < len(fields):
            pt_tuple = pt_tuple + (row[i],)
            i+=1
        point_list.append(pt_tuple)
    # Connect to database
    bulkInsert(point_list, "ET_BestMethod"+str(n))
    n +=1

['Shape@WKT', 'stations_id', 'bc0_csv_min', 'bc0_csv_max', 'bc0_csv_mean', 'bc0_csv_lat', 'bc0_csv_p', 'bc0_csv_et']
bc0_csv_min FLOAT,  bc0_csv_max FLOAT,  bc0_csv_mean FLOAT,  bc0_csv_lat FLOAT,  bc0_csv_p FLOAT, bc0_csv_et FLOAT)
['Shape@WKT', 'stations_id', 'bc1_csv_min', 'bc1_csv_max', 'bc1_csv_mean', 'bc1_csv_lat', 'bc1_csv_p', 'bc1_csv_et']
bc1_csv_min FLOAT,  bc1_csv_max FLOAT,  bc1_csv_mean FLOAT,  bc1_csv_lat FLOAT,  bc1_csv_p FLOAT, bc1_csv_et FLOAT)
['Shape@WKT', 'stations_id', 'bc2_csv_min', 'bc2_csv_max', 'bc2_csv_mean', 'bc2_csv_lat', 'bc2_csv_p', 'bc2_csv_et']
bc2_csv_min FLOAT,  bc2_csv_max FLOAT,  bc2_csv_mean FLOAT,  bc2_csv_lat FLOAT,  bc2_csv_p FLOAT, bc2_csv_et FLOAT)
['Shape@WKT', 'stations_id', 'bc3_csv_min', 'bc3_csv_max', 'bc3_csv_mean', 'bc3_csv_lat', 'bc3_csv_p', 'bc3_csv_et']
bc3_csv_min FLOAT,  bc3_csv_max FLOAT,  bc3_csv_mean FLOAT,  bc3_csv_lat FLOAT,  bc3_csv_p FLOAT, bc3_csv_et FLOAT)
['Shape@WKT', 'stations_id', 'bc4_csv_min', 'bc4_csv_max', 'bc4_csv_